import libraries / modules

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import timedelta

load & prepare data

In [2]:
faults_raw = pd.read_csv("../data/J1939Faults.csv")
diagnostics_raw = pd.read_csv("../data/vehiclediagnosticonboarddata.csv")
display(faults_raw.head())
display(diagnostics_raw.head())

/var/folders/1c/yvl84p5d2rv0n8dkqqjgmkdm0000gn/T/ipykernel_69422/3477976821.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults_raw = pd.read_csv("../data/J1939Faults.csv")


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [3]:
# prepare faults
faults_drop_cols = ["actionDescription", "activeTransitionCount", "eventDescription", "ecuSource", "ecuSoftwareVersion", "ecuModel", "ecuMake", "faultValue", "MCTNumber", "LocationTimeStamp"]
faults = faults_raw.drop(columns=faults_drop_cols)
print("\n\n--------FAULTS_RAW COLUMNS--------")
display(faults.columns)
print("\n\n--------SHAPE OF FAULTS--------")
display(faults.shape)
print("\n\n--------NaNs--------")
display(faults.isna().sum())



--------FAULTS_RAW COLUMNS--------


Index(['RecordID', 'ESS_Id', 'EventTimeStamp', 'ecuSerialNumber', 'spn', 'fmi',
       'active', 'EquipmentID', 'Latitude', 'Longitude'],
      dtype='object')



--------SHAPE OF FAULTS--------


(1187335, 10)



--------NaNs--------


RecordID                0
ESS_Id                  0
EventTimeStamp          0
ecuSerialNumber    343017
spn                     0
fmi                     0
active                  0
EquipmentID             0
Latitude                0
Longitude               0
dtype: int64

In [4]:
# join diagnostics
print("--------NaNs--------")
print(diagnostics_raw.isna().sum())
n_ids = len(diagnostics_raw['Id'])
n_unique_id = diagnostics_raw['Id'].nunique()
n_un_faults = diagnostics_raw['FaultId'].nunique()

diagnostics_raw["Value"] = diagnostics_raw["Value"].replace(
    {"FALSE": False, "TRUE": True}
)

# pivot diagnostics to long format
diagnostics = diagnostics_raw.pivot(
    index="FaultId", columns="Name", values="Value"
)

print(f"\nlen(Id): {n_ids}", f"\nN unique_Id: {n_unique_id}")
print("\n--------RECORD ID vs FAULT ID--------")
print(f"n_unique FaultID: {n_un_faults}", f"\nn_unique RecordID: {faults['RecordID'].nunique()}")
joined = faults.merge(diagnostics, how = "inner", left_on='RecordID', right_on='FaultId')
print("\n\n--------JOINED--------")
display(joined.head())
print("\n\n--------JOINED COLUMNS---------")
print(joined.columns)


--------NaNs--------
Id         0
Name       0
Value      0
FaultId    0
dtype: int64

len(Id): 12821626 
N unique_Id: 12821626

--------RECORD ID vs FAULT ID--------
n_unique FaultID: 1187335 
n_unique RecordID: 1187335


--------JOINED--------


,RecordID,ESS_Id,EventTimeStamp,ecuSerialNumber,spn,fmi,active,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,unknown,111,17,True,1439,38.857638,-84.626851,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,unknown,629,12,True,1439,38.857638,-84.626851,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,unknown,1807,2,False,1369,41.421250,-87.767361,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,unknown,1807,2,True,1369,41.421018,-87.767361,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,13063430,4364,17,False,1674,38.416481,-89.442638,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN




--------JOINED COLUMNS---------
Index(['RecordID', 'ESS_Id', 'EventTimeStamp', 'ecuSerialNumber', 'spn', 'fmi',
       'active', 'EquipmentID', 'Latitude', 'Longitude', 'AcceleratorPedal',
       'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed',
       'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad',
       'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure'],
      dtype='object')


filter out faults near service stations

In [5]:
joined_pre_station_filter = joined

In [6]:
# filter out near service stations
joined_pre_station_filter = joined
print("Labeling faults near service stations...")
stations = pd.DataFrame(
    {
        "lat": [36.0666667, 35.5883333, 36.1950],
        "lon": [-86.4347222, -86.4438888, -83.174722],
    }
)
threshold_miles = 0.5
threshold_meters = threshold_miles * 1609.34
# create geodataframes with geopandas
gdf_joined = gpd.GeoDataFrame(
    joined,
    geometry=gpd.points_from_xy(joined.Latitude, joined.Longitude),
    crs="EPSG:4326",  # WGS84 coord ref sys (lat/lon)
)
gdf_stations = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations.lat, stations.lon),
    crs="EPSG:4326",
)
target_crs = "EPSG:9311"
# reproject onto new crs for better distance measurement
gdf_joined_proj = gdf_joined.to_crs(target_crs)
gdf_stations_proj = gdf_stations.to_crs(target_crs)
# create buffers around stations
station_buf = gdf_stations_proj.geometry.buffer(threshold_meters)
combined_buffer = (
    station_buf.union_all()
)  # turns into single geometry which helps with efficiency
is_within = gdf_joined_proj.geometry.within(combined_buffer)
joined["nearStation"] = is_within.values
joined_post_filter = joined[~joined["nearStation"]]
print("\nDone! \nFaults within 1km of service station labeled in 'joined'.")
print(
    f"When filtered, this removes {len(joined_pre_station_filter['RecordID']) - len(joined_post_filter['RecordID'])} rows"
)
# filter out active=False
joined_active = joined[joined["active"]]
print(
    f"\nNumber of rows after filtering active=False out: {len(joined_active['active'])}"
)
print(
    f"Rows removed: {len(joined_pre_station_filter['RecordID']) - len(joined_active['active'])}"
)



Labeling faults near service stations...

Done! 
Faults within 1km of service station labeled in 'joined'.
When filtered, this removes 129284 rows

Number of rows after filtering active=False out: 608454
Rows removed: 578881


In [7]:
print(joined['spn'].value_counts(), joined.columns)

spn
111      365489
929      256541
96        90041
829       87788
639       41062
          ...  
46262         1
54478         1
677           1
56503         1
6327          1
Name: count, Length: 450, dtype: int64 Index(['RecordID', 'ESS_Id', 'EventTimeStamp', 'ecuSerialNumber', 'spn', 'fmi',
       'active', 'EquipmentID', 'Latitude', 'Longitude', 'AcceleratorPedal',
       'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed',
       'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad',
       'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure', 'nearStation'],
      dtype='object')


In [8]:
full_derates_raw = joined[joined['spn'] == 5246]
partial_derates_raw = joined[(joined['spn'] == 1569) & (joined['fmi'] == 31)]
# print("--------FULL DERATES---------")
# display(f"derate shape: {full_derates_raw.shape}", full_derates_raw.head(3))
#print("\n--------PARTIAL DERATES---------")
#display(f"partial derate shape: {partial_derates_raw.shape}", partial_derates_raw.head(3))

- [ ] look at time series by Equipment ID

In [9]:
col_order = [
    "RecordID",
    "EquipmentID",
    "EventTimeStamp",
    "spn",
    "fmi",
    "active",
    "Latitude",
    "Longitude",
    "AcceleratorPedal",
    "BarometricPressure",
    "CruiseControlActive",
    "CruiseControlSetSpeed",
    "DistanceLtd",
    "EngineCoolantTemperature",
    "EngineLoad",
    "EngineOilPressure",
    "EngineOilTemperature",
    "EngineRpm",
    "EngineTimeLtd",
    "FuelLevel",
    "FuelLtd",
    "FuelRate",
    "FuelTemperature",
    "IgnStatus",
    "IntakeManifoldTemperature",
    "LampStatus",
    "ParkingBrake",
    "ServiceDistance",
    "Speed",
    "SwitchedBatteryVoltage",
    "Throttle",
    "TurboBoostPressure",
    "nearStation",
    "ESS_Id",
    "ecuSerialNumber",
]
joined["EventTimeStamp"] = pd.to_datetime(joined["EventTimeStamp"])
joined = joined[col_order]
joined = joined.sort_values(["EquipmentID", "EventTimeStamp"])#.fillna(np.nan)
joined

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,Latitude,Longitude,AcceleratorPedal,BarometricPressure,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,nearStation,ESS_Id,ecuSerialNumber
33355,34467,301,2015-04-26 06:16:16,190,0,True,36.066527,-86.435277,NaN,NaN,...,4351,NaN,NaN,NaN,NaN,NaN,NaN,True,2092800,unknown
35080,36192,301,2015-04-28 05:29:21,190,0,True,36.066666,-86.435000,NaN,NaN,...,4351,NaN,NaN,NaN,NaN,NaN,NaN,True,2123154,unknown
47186,48298,301,2015-05-10 07:11:34,639,2,True,36.067083,-86.433842,8.8,14.5,...,1279,False,NaN,4.728246,3276.75,8.8,0.87,True,2346346,unknown
47213,48325,301,2015-05-10 07:59:25,639,2,False,35.586805,-86.443472,NaN,NaN,...,1279,NaN,NaN,NaN,NaN,NaN,NaN,True,2346621,unknown
48303,49415,301,2015-05-11 13:11:20,639,2,True,36.189398,-82.795601,14.4,13.92,...,1279,False,NaN,4.475814,3276.75,14.4,0.58,False,2363162,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4494,4495,R1762,2015-02-24 16:24:05,1761,18,False,39.551851,-86.045925,NaN,NaN,...,1023,NaN,NaN,NaN,NaN,NaN,NaN,False,1052146,79751302
6438,6439,R1762,2015-02-26 13:12:11,5848,9,True,39.952870,-81.936990,0,14.21,...,17407,False,NaN,2.058292,3276.75,0,0.29,False,1089561,79751302
6446,6447,R1762,2015-02-26 13:50:59,5848,9,False,39.953379,-81.937407,NaN,NaN,...,1023,NaN,NaN,NaN,NaN,NaN,NaN,False,1090499,79751302
4952,4953,R1764,2015-02-25 06:08:43,639,2,True,36.001296,-86.501435,21.6,14.4275,...,1279,False,NaN,4.378725,3276.75,0,1.74,False,1059704,unknown


In [10]:
state_cols = [
    "active",
    "CruiseControlActive",
    "IgnStatus",
    "nearStation",
    "LampStatus",
    "ParkingBrake",
]
sensor_cols = [
    "AcceleratorPedal",
    "BarometricPressure",
    "CruiseControlSetSpeed",
    "DistanceLtd",
    "EngineCoolantTemperature",
    "EngineLoad",
    "EngineOilPressure",
    "EngineOilTemperature",
    "EngineRpm",
    "EngineTimeLtd",
    "FuelLevel",
    "FuelLtd",
    "FuelRate",
    "FuelTemperature",
    "IntakeManifoldTemperature",
    "ServiceDistance",
    "Speed",
    "SwitchedBatteryVoltage",
    "Throttle",
    "TurboBoostPressure",
]

In [12]:
target_spn = 5426
joined["EventTimeStamp"] = pd.to_datetime(joined["EventTimeStamp"]).copy()
joined = joined[col_order]
joined_sorted = joined.sort_values(
    by=["EquipmentID", "EventTimeStamp"], ascending=[True, False]
).reset_index(drop=True)
time_window = pd.Timedelta(hours=2)

# 1. Create a Series containing only the timestamps of trigger events
trigger_timestamps_only = joined_sorted["EventTimeStamp"].where(
    joined_sorted["spn"] == target_spn
)
# 2. For each row, find the timestamp of the *next* trigger event within its group
#    Group by EquipmentID and use backward fill (bfill)
#    This fills NaT values with the next valid timestamp in the group
joined_sorted["next_trigger_time"] = trigger_timestamps_only.groupby(
    joined_sorted["EquipmentID"]
).bfill()
# 3. Calculate the start of the 2-hour window before the next trigger
joined_sorted["window_start_time"] = (
    joined_sorted["next_trigger_time"] - time_window
)

# 4. Label rows as True if their timestamp falls within the window:
#    [window_start_time, next_trigger_time]
#    Also ensure that a next_trigger_time actually exists (it's not NaT)
joined_sorted["derate_window"] = (
    (joined_sorted["EventTimeStamp"] >= joined_sorted["window_start_time"])
    & (joined_sorted["EventTimeStamp"] <= joined_sorted["next_trigger_time"])
    & (joined_sorted["next_trigger_time"].notna())
)
# 5. Clean up temporary columns
result_df = joined_sorted.drop(
    columns=["next_trigger_time", "window_start_time"]
)
# Display some results for verification
print(
    result_df[["EquipmentID", "EventTimeStamp", "spn", "derate_window"]].head(
        10
    )
)
print("\nValue counts for 'derate_window':")
print(result_df["derate_window"].value_counts())

# Example check: Find a trigger and look at preceding rows
trigger_indices = result_df[result_df["spn"] == target_spn].index
if len(trigger_indices) > 0:
    example_index = trigger_indices[0]
    print(f"\nExample around first trigger (index {example_index}):")
    print(
        result_df.iloc[max(0, example_index - 5) : example_index + 2][
            ["EquipmentID", "EventTimeStamp", "spn", "derate_window"]
        ]
    )

# print(
#     result_df[result_df["EquipmentID"] == "1524"]
#     .sort_values(by=["EquipmentID", "EventTimeStamp"], ascending=[True, False])
#     .reset_index(drop=True)[
#         ["EquipmentID", "EventTimeStamp", "spn", "derate_window"]
#     ]
#     .head(50)
# )
# You can now use result_df
# joined = result_df # Optional: overwrite original variable


  EquipmentID      EventTimeStamp   spn  derate_window
0         301 2020-02-21 09:00:49   653          False
1         301 2020-02-20 12:11:55  2863          False
2         301 2020-02-20 12:09:01  2863          False
3         301 2020-02-18 11:18:18  1231          False
4         301 2020-02-18 11:13:23  1231          False
5         301 2020-02-18 11:05:25   639          False
6         301 2020-02-18 11:03:09   653          False
7         301 2020-02-18 11:02:29   639          False
8         301 2020-02-16 09:02:10    96          False
9         301 2020-02-16 09:02:10   829          False

Value counts for 'derate_window':
derate_window
False    1187335
Name: count, dtype: int64
